# Align2D — All Features

Comprehensive demo of Align2D with a 0°/90° rotation pair of a perovskite crystal: auto-alignment, colormaps, scale bar, max_shift, padding, and manual mode.

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [ ]:
import numpy as np
from scipy.ndimage import shift as ndi_shift
from quantem.widget import Align2D, Show2D

rng = np.random.default_rng(42)
N = 512

# Perovskite crystal (SrTiO3-like [001] HAADF-STEM simulation)
y, x = np.mgrid[:N, :N]
img = np.zeros((N, N), dtype=np.float32)
a = 16       # lattice parameter (px)
sig_a = 2.2  # A-site column width (Sr, heavy)
sig_b = 1.8  # B-site column width (Ti, lighter)

# A-site columns (bright) at lattice corners
for i in range(-1, N // a + 2):
    for j in range(-1, N // a + 2):
        cx, cy = i * a, j * a
        if -a < cx < N + a and -a < cy < N + a:
            img += np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sig_a**2))

# B-site columns (dimmer) at body center
for i in range(-1, N // a + 2):
    for j in range(-1, N // a + 2):
        cx, cy = i * a + a / 2, j * a + a / 2
        if -a < cx < N + a and -a < cy < N + a:
            img += 0.55 * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sig_b**2))

# Defects: A-site vacancies (missing bright columns)
for vi, vj in [(4, 7), (9, 14), (14, 5), (7, 21), (19, 17)]:
    vx, vy = vi * a, vj * a
    img -= 0.9 * np.exp(-((x - vx)**2 + (y - vy)**2) / (2 * (sig_a + 0.3)**2))

# Defects: anti-site (heavy atom on B-site, anomalously bright)
for ai, aj in [(6, 11), (16, 8)]:
    cx, cy = ai * a + a / 2, aj * a + a / 2
    img += 0.35 * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sig_a**2))

# Edge dislocation strain field
disl_x, disl_y = N * 0.65, N * 0.4
theta = np.arctan2(y - disl_y, x - disl_x)
r_d = np.sqrt((x - disl_x)**2 + (y - disl_y)**2) + 1
img += 0.12 * np.sin(theta) / np.sqrt(r_d / 20)

# Specimen envelope
r = np.sqrt((x - N / 2)**2 + (y - N / 2)**2)
img *= np.exp(-np.maximum(r - N * 0.38, 0)**2 / (2 * 35**2))

# Scan noise
img += rng.normal(0, 0.04, (N, N)).astype(np.float32)
img = img.astype(np.float32)

# 0° / 90° rotation pair (small drift)
rotation_deg = 90
img_0 = img.copy()
img_90 = np.rot90(img, k=1).copy()
img_90 = ndi_shift(img_90, (3.7, -2.1), order=3, mode='reflect').astype(np.float32)
img_90 += rng.normal(0, 0.04, (N, N)).astype(np.float32)

# Rotation-corrected version for alignment
img_90_corrected = np.rot90(img_90, k=-1)

# Large-drift variant (for max_shift demo)
img_90_large = np.rot90(img, k=1).copy()
img_90_large = ndi_shift(img_90_large, (15.3, -8.7), order=3, mode='reflect').astype(np.float32)
img_90_large += rng.normal(0, 0.04, (N, N)).astype(np.float32)
img_90_large_corrected = np.rot90(img_90_large, k=-1)

print(f"Image 0°:  {img_0.shape}")
print(f"Image 90°: {img_90.shape}")
print(f"Rotation: {rotation_deg}°")

## Original images (Show2D)

In [ ]:
Show2D([img_0, img_90], title="0° / 90° acquisition pair", labels=["0°", "90°"])

## 1. Default — phase correlation auto-alignment

Correct the known 90° rotation, then auto-detect sub-pixel translation via phase correlation with Tukey windowing and matrix DFT refinement.

In [ ]:
w1 = Align2D(
    img_0, img_90_corrected,
    title="Auto-alignment (phase correlation)",
    label_a="0° (ref)", label_b="90° (corrected)",
)
w1

In [ ]:
dx, dy = w1.offset
print(f"Rotation: {rotation_deg}°")
print(f"Translation: dx={dx:.2f}, dy={dy:.2f}")
print(f"NCC: {w1.xcorr_zero:.4f} (before) → {w1.ncc_aligned:.4f} (after)")

## 2. Viridis colormap + larger canvas

In [ ]:
Align2D(
    img_0, img_90_corrected,
    title="Viridis + large canvas",
    cmap="viridis",
    canvas_size=400,
    label_a="0° (ref)", label_b="90° (corrected)",
)

## 3. With scale bar (calibrated pixel size)

SrTiO3 lattice parameter ≈ 3.9 Å → pixel size = 3.9/16 ≈ 0.24 nm.

In [ ]:
Align2D(
    img_0, img_90_corrected,
    title="With scale bar",
    pixel_size=0.24,
    label_a="0° (ref)", label_b="90° (corrected)",
)

## 4. Larger shift with max_shift constraint

For periodic crystals, cross-correlation can match at any lattice spacing. Use `max_shift` to constrain the search to the correct peak.

In [ ]:
Align2D(
    img_0, img_90_large_corrected,
    title="Constrained search (max_shift=25)",
    max_shift=25.0,
    label_a="0° (ref)", label_b="90° large drift",
)

## 5. Custom opacity + extra padding

In [ ]:
Align2D(
    img_0, img_90_corrected,
    title="Custom opacity + padding",
    opacity=0.7,
    padding=0.4,
    cmap="inferno",
    label_a="0° (ref)", label_b="90° (corrected)",
)

## 6. Auto-align disabled (manual only)

In [ ]:
Align2D(
    img_0, img_90_corrected,
    title="Manual alignment (auto_align=False)",
    auto_align=False,
    label_a="0° (ref)", label_b="90° (corrected)",
)

## 7. Bounded shift (max_shift=10)

Limit the maximum allowed translation to ±10 pixels. The AlignPad and drag are clamped to this range.

In [ ]:
Align2D(
    img_0, img_90_corrected,
    title="Bounded shift (max_shift=10)",
    max_shift=10.0,
    label_a="0° (ref)", label_b="90° (corrected)",
)

## Interactive controls (all widgets)

Every Align2D widget includes these interactive controls:

- **Fine** toggle — Narrows the AlignPad joystick range to ±5 px for sub-pixel precision adjustments.
- **Panels** toggle — Show/hide the side-by-side comparison panels.
- **FFT** toggle — Show FFT magnitude of both images.